# Project Solution

In [1]:
from datetime import datetime

In [18]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    
    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message    
        
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f"EXCEPTION: {datetime.utcnow().isoformat()}: {exception}")
        
    

In [14]:
ex1 = WidgetException("Some custom message", 10, 100)
ex2 = WidgetException(customer_message="A custom user message.")

In [15]:
ex1.log_exception()

EXCEPTION: 2023-07-02T14:06:58.141243: {'type': 'WidgetException', 'message': 'Some custom message', 'args': ()}


In [16]:
ex2.log_exception()

EXCEPTION: 2023-07-02T14:07:19.405296: {'type': 'WidgetException', 'message': 'Generic Widget exception.', 'args': ()}


In [17]:
class SupplierException(WidgetException):
    message = "Supplier exception."
    
class NotManufacturedException(SupplierException):
    message = "Widget is no longer manufactured by supplier."
    
class ProductionDelayedException(SupplierException):
    message = "Widget production has been delayed by manufacturer."
    
class ShippingDelayedException(SupplierException):
    message = "Widget shipping has been delayed by supplier."
    
class CheckoutException(WidgetException):
    message = "Checkout exception"
    
class InventoryException(CheckoutException):
    message = "Checkout inventory exception"
     
class OutOfStockException(InventoryException):
    message = "Inventory out of stock"
    
class PricingException(CheckoutException):
    message = "Checkout pricing exception"
    
class InvalidCouponCodeException(PricingException):
    message = "Invalid checkout coupon code."
    
class CannotStackCouponException(PricingException):
    message = "Cannot stack checkout coupon code."

In [19]:
try:
    raise CannotStackCouponException()
except CannotStackCouponException as ex:
    ex.log_exception()
    raise

EXCEPTION: 2023-07-02T14:17:59.969842: {'type': 'CannotStackCouponException', 'message': 'Cannot stack checkout coupon code.', 'args': ()}


CannotStackCouponException: ()

In [21]:
from http import HTTPStatus
import json

In [24]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    
    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message    
        
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:]
        }
        print(f"EXCEPTION: {datetime.utcnow().isoformat()}: {exception}")
        
    def to_json(self):
        response = {
            "code": self.http_status.value,
            "message": f"{self.http_status.phrase}: {self.customer_message}",
            "category": type(self).__name__,
            "time_utc": datetime.utcnow().isoformat()

        }
        return json.dumps(response)

In [25]:
e = WidgetException("same custom message for log and user")
e.log_exception()

EXCEPTION: 2023-07-02T14:23:50.991757: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': ()}


In [32]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: same custom message for log and user',
 'category': 'WidgetException',
 'time_utc': '2023-07-02T14:24:43.698369'}

In [33]:
e = WidgetException("same custom message for log and user", 
                    customer_message="custom user message")
e.log_exception()

EXCEPTION: 2023-07-02T14:25:39.497799: {'type': 'WidgetException', 'message': 'same custom message for log and user', 'args': ()}


In [34]:
json.loads(e.to_json())

{'code': 500,
 'message': 'Internal Server Error: custom user message',
 'category': 'WidgetException',
 'time_utc': '2023-07-02T14:25:50.478812'}

In [36]:
try:
    raise WidgetException("custom error message")
except WidgetException as ex:
    print(repr(ex.__traceback__))

In [37]:
import traceback

In [38]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException("custom message")
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        print(list(tb.format()))

['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_30163/629708075.py", line 2, in <cell line: 1>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_30163/629708075.py", line 5, in <cell line: 1>\n    raise WidgetException("custom message")\n', "WidgetException: ('custom message',)\n"]


In [39]:
try:
    raise ValueError
except ValueError:
    try:
        raise WidgetException("custom message")
    except WidgetException as ex:
        tb = traceback.TracebackException.from_exception(ex)
        tb = "".join(tb.format())
        print(tb)

Traceback (most recent call last):
  File "/tmp/ipykernel_30163/3472716484.py", line 2, in <cell line: 1>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_30163/3472716484.py", line 5, in <cell line: 1>
    raise WidgetException("custom message")
WidgetException: ('custom message',)



In [40]:
try:
    raise ValueError
except ValueError:
    raise WidgetException("custom message")
   

WidgetException: ('custom message',)

In [43]:
class WidgetException(Exception):
    message = 'Generic Widget exception.'
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    
    def __init__(self, *args, customer_message=None):
        super().__init__(args)
        if args:
            self.message = args[0]
        self.customer_message = customer_message if customer_message is not None else self.message    
        
    @property
    def traceback(self):
        tb = traceback.TracebackException.from_exception(self)
        tb = tb.format()
        return tb    
    
    
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "message": self.message,
            "args": self.args[1:],
            "traceback": list(self.traceback)
        }
        print(f"EXCEPTION: {datetime.utcnow().isoformat()}: {exception}")
        
    def to_json(self):
        response = {
            "code": self.http_status.value,
            "message": f"{self.http_status.phrase}: {self.customer_message}",
            "category": type(self).__name__,
            "time_utc": datetime.utcnow().isoformat()

        }
        return json.dumps(response)

In [49]:
try:
    raise WidgetException()
except WidgetException as ex:
    ex.log_exception()
    print("-"*10)
    print(ex.to_json())

EXCEPTION: 2023-07-02T14:35:05.361793: {'type': 'WidgetException', 'message': 'Generic Widget exception.', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_30163/1966510761.py", line 2, in <cell line: 1>\n    raise WidgetException()\n', 'WidgetException: ()\n']}
----------
{"code": 500, "message": "Internal Server Error: Generic Widget exception.", "category": "WidgetException", "time_utc": "2023-07-02T14:35:05.361942"}


In [50]:
try:
    a = 1 / 0
except ZeroDivisionError:
    try:
        raise WidgetException()
    except WidgetException as ex:
        print("".join(ex.traceback))

Traceback (most recent call last):
  File "/tmp/ipykernel_30163/3805027978.py", line 2, in <cell line: 1>
    a = 1 / 0
ZeroDivisionError: division by zero

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_30163/3805027978.py", line 5, in <cell line: 1>
    raise WidgetException()
WidgetException: ()



In [51]:
class SupplierException(WidgetException):
    message = "Supplier exception."
    
class NotManufacturedException(SupplierException):
    message = "Widget is no longer manufactured by supplier."
    
class ProductionDelayedException(SupplierException):
    message = "Widget production has been delayed by manufacturer."
    
class ShippingDelayedException(SupplierException):
    message = "Widget shipping has been delayed by supplier."
    
class CheckoutException(WidgetException):
    message = "Checkout exception"
    
class InventoryException(CheckoutException):
    message = "Checkout inventory exception"
     
class OutOfStockException(InventoryException):
    message = "Inventory out of stock"
    
class PricingException(CheckoutException):
    message = "Checkout pricing exception"
    
class InvalidCouponCodeException(PricingException):
    message = "Invalid checkout coupon code."
    
class CannotStackCouponException(PricingException):
    message = "Cannot stack checkout coupon code."

In [53]:
e = InvalidCouponCodeException("User tried to pull a fast one on us.",
                              customer_message="Sorry. This coupon is not valid.")

In [54]:
e.log_exception()

EXCEPTION: 2023-07-02T14:38:01.998072: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us.', 'args': (), 'traceback': ["InvalidCouponCodeException: ('User tried to pull a fast one on us.',)\n"]}


In [55]:
e.to_json()

'{"code": 500, "message": "Internal Server Error: Sorry. This coupon is not valid.", "category": "InvalidCouponCodeException", "time_utc": "2023-07-02T14:38:22.066310"}'

In [56]:
try:
    raise ValueError
except ValueError:
    try:
        raise InvalidCouponCodeException("User tried to pull a fast one on us.",
                              customer_message="Sorry. This coupon is not valid.")
    except InvalidCouponCodeException as ex:
        ex.log_exception()
        print("-"*10)
        print(ex.to_json())
        print("-"*10)
        print("".join(ex.traceback))

EXCEPTION: 2023-07-02T14:40:28.201629: {'type': 'InvalidCouponCodeException', 'message': 'User tried to pull a fast one on us.', 'args': (), 'traceback': ['Traceback (most recent call last):\n', '  File "/tmp/ipykernel_30163/3126969583.py", line 2, in <cell line: 1>\n    raise ValueError\n', 'ValueError\n', '\nDuring handling of the above exception, another exception occurred:\n\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_30163/3126969583.py", line 5, in <cell line: 1>\n    raise InvalidCouponCodeException("User tried to pull a fast one on us.",\n', "InvalidCouponCodeException: ('User tried to pull a fast one on us.',)\n"]}
----------
{"code": 500, "message": "Internal Server Error: Sorry. This coupon is not valid.", "category": "InvalidCouponCodeException", "time_utc": "2023-07-02T14:40:28.201800"}
----------
Traceback (most recent call last):
  File "/tmp/ipykernel_30163/3126969583.py", line 2, in <cell line: 1>
    raise ValueError
ValueError

During handling